### Import Libraries

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
import os.path
import math
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import pickle
from joblib import dump, load

### Load pre-computed stats

In [4]:
df_train = pd.read_pickle(os.path.abspath('') + '\\..\\Derived stats\\match_stats_train.pkl')
df_test = pd.read_pickle(os.path.abspath('') + '\\..\\Derived stats\\match_stats_test.pkl')

# delete outlier
""" Assuming, in most normal matches, teams 
score more than 75 runs on an average as shown
by past statistics. """
df_train = df_train[df_train['Runs'] > 75]
df_test = df_test[df_test['Runs'] > 75]

cols = df_train.columns
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, 'utf-8')) else x)
df_train.columns = cols
df_test.columns = cols

# To remove the other inning for which outliers were removed
match_ids = df_test['Match_ID'].unique()
m_ids_toBeRemoved = []          
for m_id in match_ids:
    if len(df_test[df_test['Match_ID'] == m_id]) !=2:
        m_ids_toBeRemoved.append(m_id)

for m_id in m_ids_toBeRemoved:
    df_test = df_test[df_test['Match_ID'] != m_id]

### Computing additional features

#### Adding feature: Average runs scored against same opponent in previous matches

In [6]:
""" Previous run with same opponent """

prev_run = []
for Date, Team_Name, Opp_Team  in df_train[['Date','Team_Name', 'Opp_Team']].itertuples(index=False):
    df_t = df_train.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team & Date < @Date")['Runs']
    if len(df_t) == 0:
        prev_run.append(df_train.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team")['Runs'].mean())
        continue
    prev_run.append(df_train.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team & Date < @Date")['Runs'].values[-1])
df_train['Prev Run'] = prev_run

prev_run = []
for Date, Team_Name, Opp_Team  in df_test[['Date','Team_Name', 'Opp_Team']].itertuples(index=False):
    df_t = df_test.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team & Date < @Date")['Runs']
    if len(df_t) == 0:
        df_t_t = df_train.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team & Date < @Date")
        if len(df_t_t) == 0:
            prev_run.append(0)
        else:
            prev_run.append(df_t_t['Runs'].values[-1])
        continue
    prev_run.append(df_test.query("Team_Name == @Team_Name & Opp_Team == @Opp_Team & Date < @Date")['Runs'].values[-1])
df_test['Prev Run'] = prev_run

#### Adding feature: Average runs scored in a venue till date irrespective of teams

In [7]:
""" Previous run on venue irrespective of team """

prev_run = []
for Date, city  in df_train[['Date', 'City']].itertuples(index=False):
    df_t = df_train.query("City == @city & Date < @Date")
    if len(df_t) == 0:
        df_t_t = df_train.query("City == @city")
        if len(df_t_t) == 0:
            prev_run.append(0)
        else:
            prev_run.append(df_t_t['Runs'].mean())
        continue
    prev_run.append(df_t['Runs'].mean())
df_train['Prev Venue Run'] = prev_run

prev_run = []
for Date, city  in df_test[['Date', 'City']].itertuples(index=False):
    df_te = df_test.query("City == @city & Date < @Date")
    df_tr = df_train.query("City == @city & Date < @Date")
    prev_run.append((df_te['Runs'].sum(axis = 0, skipna = True) + df_tr['Runs'].sum(axis = 0, skipna = True))/(len(df_te)+len(df_tr)))
df_test['Prev Venue Run'] = prev_run

### Print data before model fitting

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

### Selecting Features to train the model

In [9]:
df_train_inning_1 = df_train[df_train['Innnings'] == 1]
df_test_inning_1 = df_test[df_test['Innnings'] == 1]

In [10]:
df_inning_2 = df_test[df_test['Innnings'] == 2]
inning_2_input = df_inning_2[['Avg_Bat1', 'Avg_Bat2', 'Avg_Bat3', 'Avg_Bat4', 'Avg_Bat5', 'Avg_Bat6', 'Avg_Bat7', 'Avg_Bat8', 'Avg_Bat9', 'Avg_Bat10', 'Avg_Bat11','Prev Run', 'Prev Venue Run', 'Past_lead']]

In [11]:
cols = ['Avg_Bat1', 'Avg_Bat2', 'Avg_Bat3', 'Avg_Bat4', 'Avg_Bat5', 'Avg_Bat6', 'Avg_Bat7', 'Avg_Bat8', 'Avg_Bat9', 'Avg_Bat10', 'Avg_Bat11', 'Prev Run', 'Prev Venue Run', 'Past_lead']

X_train = df_train_inning_1[cols]
y_train = df_train_inning_1[['Runs']]

X_test = df_test_inning_1[cols]
y_test = df_test_inning_1[['Runs']]

### Gradient Boosted Regressor

In [12]:
param_dict = {'max_depth': [2,3,4], 'learning_rate': np.linspace(0.01,0.1,10), 'n_estimators': np.linspace(10,200,num = 20, dtype = int)}
gradientRegressionModel = GradientBoostingRegressor()
gradientRegressionModel_cv = GridSearchCV(estimator=gradientRegressionModel, param_grid=param_dict, cv= 5, iid=False, n_jobs = -1)
gradientRegressionModel_cv.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter...
                             

In [13]:
predictedRunsTrain = gradientRegressionModel_cv.predict(X_train)
print('Train random regression model RMSE:', np.sqrt(mean_squared_error(y_train, predictedRunsTrain)))

Train random regression model RMSE: 21.395961899457035


In [14]:
predictedRunsTest = gradientRegressionModel_cv.predict(X_test)
print('Test random regression Linear model RMSE:', np.sqrt(mean_squared_error(y_test, predictedRunsTest)))

Test random regression Linear model RMSE: 22.711199367815382


In [16]:
gradientRegressionModel_cv.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.06000000000000001, loss='ls',
                          max_depth=2, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=60,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Accuracy of winner prediction

In [17]:
matches = pd.read_csv('matches.csv')
matchId_winnerInning_dict = {}
for index, row in matches.iterrows():
    if row['winner'] == row['team1']:
        matchId_winnerInning_dict[row['id']] = 1
    if row['winner'] == row['team2']:
        matchId_winnerInning_dict[row['id']] = 2

In [20]:
inning1_out = predictedRunsTest
inning2_out = gradientRegressionModel_cv.predict(inning_2_input)

winner_df = df_inning_2[['Match_ID']]
winner = [ 1 if inning1_out[i] > inning2_out[i] else 2 for i in range(len(inning1_out))]
winner_df['Winner'] = winner
correct_count = 0
for match_id, winner  in winner_df[['Match_ID', 'Winner']].itertuples(index=False):
    if winner == matchId_winnerInning_dict[match_id]:
        correct_count += 1
winPredictionAccuracy = float((correct_count*100)/len(winner_df))

print(winPredictionAccuracy)

77.77777777777777


C:\Users\Mohinish\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Print Scores prediction

In [ ]:
## plot and see the data..
for id, runs in df_test[['Match_ID', 'Runs']].itertuples(index=False):
    print(runs)

In [ ]:
for runs in predictedRunsTest:
    print(runs)